In [1]:
##Libraries##
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import datetime

In [17]:
##Connection to cmv_uat database##

server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [21]:
##tradedate = '2021-07-03'  ###for risk Calcs###
query = "SELECT * FROM [dbo].[fxblotter.t]"
# Read the SQL query into a DataFrame
fxblotter = pd.read_sql(query, connection, index_col='tradeid', parse_dates=True)
fxblotter.sort_index(inplace=True)
#datetime format#
fxblotter['tradedate'] = pd.to_datetime(fxblotter['tradedate'])
fxblotter['valuedate'] = pd.to_datetime(fxblotter['valuedate'])
## filtro para fecha en T o dia de consulta ### 
##fxblottert = fxblotter.loc[fxblotter['tradedate'] == tradedate]
## filtro para accionid, escencialmente deja los Lost deals por fuera del calculo ##
fxblottert = fxblotter
fxblottert = fxblottert[fxblottert['accionid'] == 1] ### acepting 'addnew', 'lost' do not count for spread analysis##

## Weighted average 
bidfxblottert = fxblottert.loc[fxblotter['comccyid'] == 1 ]
askfxblottert = fxblottert.loc[fxblotter['venccyid'] == 1 ] 
spreadbidmpp =  bidfxblottert['monex_pp'] - fxblottert['px_sp'] 
spreadaskmpp =  askfxblottert['px_sp'] - fxblottert['monex_pp']
#bidclientpp = fxblottert.loc[fxblottert['comccyid'] == 1, 'montoven'].sum() / bidtotalcliente
#askclientpp = fxblottert.loc[fxblottert['venccyid'] == 1, 'montocom'].sum() / asktotalcliente
print(spreadbidmpp.mean())
print(spreadaskmpp.mean())
##Resta o suma de las dos columnas

##sidechoice = int(sidechoiceraw)

#filtra por solo bids

##bidcoverpp = fxblottert.loc[fxblottert['comccyid'] == 1].sub(['px_cover','px_sp'],axis='px-sp') 
#askcoverpp = 


3.2121223470661646
1.3455619596541826
